## 1. Set up communication with the FR3

### 1.1 Set up communication with the arm

In [1]:
%load_ext autoreload
%autoreload 2
import sys 
from FR3Py.robot.interface import FR3Real
robot = FR3Real(robot_id='fr3')

Interface Running...


In [2]:
# Print the arm state
robot.getJointStates()

{'q': array([ 8.88668226e-04, -7.84690130e-01,  2.73541349e-03, -2.35642117e+00,
        -8.17048037e-04,  1.57105756e+00,  7.85200413e-01]),
 'dq': array([-2.51116387e-05, -1.10590476e-03, -5.65871260e-04,  6.87839623e-05,
         1.02930775e-03,  2.30923927e-04, -5.28051736e-04]),
 'T': array([-0.1355927 , -4.06793356, -0.85097349, 22.99258804,  1.06950605,
         2.08829474,  0.09478214]),
 'M': array([[ 4.83808459e-01, -1.21138078e-02,  4.81430841e-01,
         -1.11041177e-02,  5.55204690e-02, -3.78922190e-03,
          1.57051619e-03],
        [-1.21138078e-02,  1.53782493e+00,  4.35469186e-03,
         -6.90790144e-01, -1.43198603e-02, -2.38443492e-02,
         -6.84396273e-04],
        [ 4.81430841e-01,  4.35469186e-03,  9.85185106e-01,
         -2.76988347e-02,  6.82273553e-02, -4.60421616e-03,
          3.48942024e-03],
        [-1.11041177e-02, -6.90790144e-01, -2.76988347e-02,
          9.51042653e-01,  2.77232756e-02,  1.06072452e-01,
          3.14484231e-04],
        

### 1.2 Set up communication with the gripper

In [3]:
from fr3_gripper import Gripper, GripperState
gripper = Gripper("192.168.123.250")

# Initialize the gripper
gripper.homing()

# Print the gripper state
gripper_state = gripper.readOnce()
print(gripper_state)

{'width': 0.08006671071052551, 'max_width': 0.08006933331489563, 'is_grasped': False, 'temperature': 48, 'time (ms)': 191}


## 2. Define robot task parameters

### 2.1 Load PinocchioModel and define various parameters

In [4]:
import numpy as np
from FR3Py.robot.model_collision_avoidance import PinocchioModel
import time

pin_robot = PinocchioModel()
joint_lb = np.array([-2.3093, -1.5133, -2.4937, -2.7478, -2.48, 0.8521, -2.6895])
joint_ub = np.array([2.3093, 1.5133, 2.4937, -0.4461, 2.48, 4.2094, 2.6895])
torque_lb = np.array([-87, -87, -87, -87, -12, -12, -12])
torque_ub = np.array([87, 87, 87, 87, 12, 12, 12])
joint_acc_lb = np.array([-10, -10, -10, -10, -10, -10, -10])
joint_acc_ub = np.array([10, 10, 10, 10, 10, 10, 10])
v_EE_lb = np.array([-0.15, -0.15, -0.15])
v_EE_ub = np.array([0.15, 0.15, 0.15])
static_friction = np.array([0.8, 1.1, 0.5, 1.7, 1.8, 1.0, 0.6])

n_joints = 7
n_controls = 7
q_bar = 0.5*(joint_ub + joint_lb)


### 2.2 Define the obstacles using scaling functions and setup problems

In [5]:
# Obstacle parameters
obstacle_config = {
    "obs1":{
        "type": "polytope3d",
        "vertices": [[0.226, 0.293, 0.28], [0.234, 0.287, 0.28], [0.416, 0.543, 0.28], [0.424, 0.537, 0.28], [0.226, 0.293, 0.0], [0.234, 0.287, 0.0], [0.416, 0.543, 0.0], [0.424, 0.537, 0.0]],
        "pos": [0.0, 0.0, 0.0],
        "quat": [0.0, 0.0, 0.0, 1.0]
    },

    "obs2":{
        "type": "polytope3d",
        "vertices": [[0.417, 0.536, 0.28], [0.423, 0.544, 0.28], [0.047, 0.826, 0.28], [0.053, 0.834, 0.28], [0.417, 0.536, 0.0], [0.423, 0.544, 0.0], [0.047, 0.826, 0.0], [0.053, 0.834, 0.0]],
        "pos": [0.0, 0.0, 0.0],
        "quat": [0.0, 0.0, 0.0, 1.0]
    },

    "obs3":{
        "type": "polytope3d",
        "vertices": [[0.054, 0.827, 0.28], [0.046, 0.833, 0.28], [-0.136, 0.577, 0.28], [-0.144, 0.583, 0.28], [0.054, 0.827, 0.0], [0.046, 0.833, 0.0], [-0.136, 0.577, 0.0], [-0.144, 0.583, 0.0]],
        "pos": [0.0, 0.0, 0.0],
        "quat": [0.0, 0.0, 0.0, 1.0]
    },

    "obs4":{
        "type": "polytope3d",
        "vertices": [[-0.137, 0.584, 0.28], [-0.143, 0.576, 0.28], [0.233, 0.294, 0.28], [0.227, 0.286, 0.28], [-0.137, 0.584, 0.0], [-0.143, 0.576, 0.0], [0.233, 0.294, 0.0], [0.227, 0.286, 0.0]],
        "pos": [0.0, 0.0, 0.0],
        "quat": [0.0, 0.0, 0.0, 1.0]
    }

}

In [6]:
from cores.utils.bounding_shape_coef_mj import BoundingShapeCoef
from cores.obstacle_collections.polytope_collection import PolytopeCollection
import scalingFunctionsHelperPy as sfh
import HOCBFHelperPy as hh
from scipy.spatial.transform import Rotation
import multiprocessing
from liegroups import SO3
from cores.utils.proxsuite_utils import init_proxsuite_qp
from cores.utils.rotation_utils import get_quat_from_rot_matrix

# Load the bounding shape coefficients
BB_coefs = BoundingShapeCoef()
selected_BBs = ["HAND_BB", "LINK7_BB", "LINK6_BB"]
n_selected_BBs = len(selected_BBs)

# Obstacle
n_polytope = len(obstacle_config)
obs_col = PolytopeCollection(3, n_polytope, obstacle_config)

# CBF parameters
obstacle_kappa = 80

robot_SFs = []
for (i, bb_key) in enumerate(selected_BBs):
    ellipsoid_quadratic_coef = BB_coefs.coefs[bb_key]
    SF_rob = sfh.Ellipsoid3d(True, ellipsoid_quadratic_coef, np.zeros(3))
    robot_SFs.append(SF_rob)

polytope_SFs = []
for (i, obs_key) in enumerate(obs_col.face_equations.keys()):
    A_obs_np = obs_col.face_equations[obs_key]["A"]
    b_obs_np = obs_col.face_equations[obs_key]["b"]
    obs_kappa = obstacle_kappa
    SF_obs = sfh.LogSumExp3d(False, A_obs_np, b_obs_np, obs_kappa)
    polytope_SFs.append(SF_obs)
    
hyperplane_SFs = []
SF_hyperplane = sfh.Hyperplane3d(False, np.array([0,0,1]), 0.0+1.0)
hyperplane_SFs.append(SF_hyperplane)

# Create the problem collection
n_threads = max(multiprocessing.cpu_count() -1, 1)
probs = hh.Problem3dCollection(n_threads)
for i in range(len(selected_BBs)):
    SF_rob = robot_SFs[i]
    frame_id = i
    for (j, obs_key) in enumerate(obs_col.face_equations.keys()):
        SF_obs = polytope_SFs[j]
        vertices = obs_col.face_equations[obs_key]["vertices_in_world"]
        prob = hh.EllipsoidAndLogSumExp3dPrb(SF_rob, SF_obs, vertices)
        probs.addProblem(prob, frame_id)

    for j in range(len(hyperplane_SFs)):
        SF_obs = hyperplane_SFs[j]
        prob = hh.EllipsoidAndHyperplane3dPrb(SF_rob, SF_obs)
        probs.addProblem(prob, frame_id)


/workspaces/HOCBF-FR3-Experiments/cores/obstacle_collections/polytope_collection.py:13: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  Z = linkage(dist_matrix, method='single')


## 3. Control the arm to starting point

In [13]:
# Rotate link 1 to reach preparation configuration
T = 6
q_d = np.array([np.pi/3, -np.pi/4, 0, -np.pi/4*3, 0, np.pi/2 , np.pi/4])


Kp_joint = np.diag([1, 1, 1, 1, 4, 2, 1])*1.5e2
Kd_joint = np.diag([2, 2, 2, 2, 2, 2, 2])*1e2
t_start = time.time()
while time.time() - t_start < T:
    robot_info = robot.getJointStates()
    q = robot_info['q'] # shape (7,)
    dq = robot_info['dq'] # shape (7,)
    M = robot_info['M'] + 0.25*np.eye(n_joints) # shape (7,7)
    G = robot_info['G'] # shape (7,)
    Coriolis = robot_info['C'] # shape (7,)

    q_pin = 0.025*np.ones(9)
    dq_pin = np.zeros(9)
    q_pin[0:n_joints] = q # shape (9,)
    dq_pin[0:n_joints] = dq # shape (9,)

    pin_info = pin_robot.getInfo(q_pin, dq_pin)
    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    M_pin = pin_info["M"][0:n_joints,0:n_joints] + 0.15*np.eye(n_joints) # shape (7,7)
    nle_pin = pin_info["nle"][0:n_joints]  # shape (7,)
    G_pin = pin_info["G"][0:n_joints]  # shape (7,)

    W = np.diag(1.0/(joint_ub-joint_lb))
    eq = W @ (q - q_d)
    deq = W @ dq
    ddq_nominal = - Kd_joint @ deq - Kp_joint @ eq

    ddq = ddq_nominal

    tau = M @ ddq + Coriolis + static_friction * np.tanh(dq)

    tau = np.clip(tau, torque_lb, torque_ub)
    robot.setCommands(tau)

robot.setCommands(np.zeros_like(tau))


## 4. Collision avoidance without circulation

### 4.1 Define the CBF-QP and various parameters

In [12]:
from cores.utils.trajectory_utils import PositionTrapezoidalTrajectory, OrientationTrapezoidalTrajectory

# Define proxuite problem
n_obstacle = n_polytope + 1
n_CBF = n_selected_BBs*n_obstacle
n_controls = 7
n_in = n_controls + n_CBF + 3
cbf_qp = init_proxsuite_qp(n_v=n_controls, n_eq=0, n_in=n_in)

# Define CBF parameters
alpha0 = 1.03
gamma1 = 20
gamma2 = 20
compensation = 0

# Define tracking gains
K_p_pos = np.diag([100,100,100])*0.9
K_d_pos = np.diag([50,50,50])*0.5

K_p_rot = np.diag([200,200,200])*1.8
K_d_rot = np.diag([100,100,100])*0.4

Kp_joint = 20*np.diag([1, 1, 1, 1, 1, 1, 1])
Kd_joint = 10*np.diag([1, 1, 1, 1, 1, 1, 1])

# Define the record
ddq_list = []
tau_list = []
all_h_list = []

### 4.2 Move from the starting point to pre-grasping pose 

In [13]:
t_final = 10

# Get current pose
robot_info = robot.getJointStates()
q = robot_info['q'] # shape (7,)
dq = robot_info['dq'] # shape (7,)

q_pin = 0.30*np.ones(9)
dq_pin = np.zeros(9)
q_pin[0:n_joints] = q # shape (9,)
dq_pin[0:n_joints] = dq # shape (9,)

pin_info = pin_robot.getInfo(q_pin, dq_pin)
P_EE = pin_info["P_EE"]
R_EE = pin_info["R_EE"]

# Translational trajectory
P_EE_start = P_EE.copy()
P_EE_pre_grasping = np.array([0.15, 0.62, 0.10])

via_points = np.array([P_EE_start, P_EE_pre_grasping])
target_time = np.array([0, t_final])
Ts = 0.01
traj_position = PositionTrapezoidalTrajectory(via_points, target_time, T_antp=0.2, Ts=Ts)

# Rotational trajectory
R_EE_start  = R_EE.copy()

roll = np.pi
pitch = 0
yaw = np.pi/3
R_EE_pre_grasping = Rotation.from_euler('xyz', [roll, pitch, yaw]).as_matrix()

orientations = np.array([R_EE_start, R_EE_pre_grasping])
target_time = np.array([0, t_final])
traj_orientation = OrientationTrapezoidalTrajectory(orientations, target_time, Ts=Ts)

t_start = time.time()
while time.time() - t_start < t_final + 1:
    t = time.time() - t_start
    traj_pos, traj_pos_dt, traj_pos_dtdt = traj_position.get_traj_and_ders(t)
    traj_ori, traj_ori_dt, traj_ori_dtdt = traj_orientation.get_traj_and_ders(t)

    robot_info = robot.getJointStates()
    q = robot_info['q'] # shape (7,)
    dq = robot_info['dq'] # shape (7,)
    M = robot_info['M'] + 0.20*np.eye(n_joints) # shape (7,7)
    G = robot_info['G'] # shape (7,)
    Coriolis = robot_info['C'] # shape (7,)

    q_pin = 0.025*np.ones(9)
    dq_pin = np.zeros(9)
    q_pin[0:n_joints] = q # shape (9,)
    dq_pin[0:n_joints] = dq # shape (9,)

    pin_info = pin_robot.getInfo(q_pin, dq_pin)

    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)

    # Primary obejctive: tracking control
    e_pos = P_EE - traj_pos # shape (3,)
    e_pos_dt = v_EE[:3] - traj_pos_dt # shape (3,)
    v_dt = traj_pos_dtdt - K_p_pos @ e_pos - K_d_pos @ e_pos_dt

    e_rot = SO3(R_EE @ traj_ori.T).log() # shape (3,)
    e_rot_dt = v_EE[3:] - traj_ori_dt # shape (3,)
    omega_dt = traj_ori_dtdt -K_p_rot @ e_rot - K_d_rot @ e_rot_dt

    v_EE_dt_desired = np.concatenate([v_dt, omega_dt])
    S = J_EE
    S_pinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    S_null = (np.eye(len(q)) - S_pinv @ S)
    ddq_task = S_pinv @ (v_EE_dt_desired - dJdq_EE)

    # Secondary objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    q_bar = 1/2*(joint_ub+joint_lb)
    e_joint = W @ (q - q_bar)
    e_joint_dot = W @ dq
    ddq_nominal = ddq_task + S_null @ (- Kp_joint @ e_joint - Kd_joint @ e_joint_dot)

    # CBF
    C = np.zeros([n_in, n_controls])
    lb = np.zeros(n_in)
    ub = np.zeros(n_in)
    CBF_tmp = np.zeros(n_CBF)
    phi1_tmp = np.zeros(n_CBF)
    phi2_tmp = np.zeros(n_CBF)

    all_P_np = np.zeros([n_selected_BBs, 3])
    all_quat_np = np.zeros([n_selected_BBs, 4])
    all_J_np = np.zeros([n_selected_BBs, 6, 7])
    all_dJdq_np = np.zeros([n_selected_BBs, 6])

    for (ii, bb_key) in enumerate(selected_BBs):
        all_P_np[ii, :] = pin_info["P_"+bb_key]
        all_J_np[ii, :, :] = pin_info["J_"+bb_key][:,:7]
        all_quat_np[ii, :] = get_quat_from_rot_matrix(pin_info["R_"+bb_key])
        all_dJdq_np[ii, :] = pin_info["dJdq_"+bb_key][:7]

    all_h_np, _, _, all_phi1_np, all_actuation_np, all_lb_np, all_ub_np = \
        probs.getCBFConstraints(dq, all_P_np, all_quat_np, all_J_np, all_dJdq_np, alpha0, gamma1, gamma2, compensation)
    
    # CBF-QP constraints
    C[0:n_CBF,:] = all_actuation_np
    lb[0:n_CBF] = all_lb_np
    ub[0:n_CBF] = all_ub_np
    CBF_tmp = all_h_np
    phi1_tmp = all_phi1_np

    g = -ddq_nominal

    C[n_CBF:n_CBF+n_controls,:] = np.eye(n_controls)
    lb[n_CBF:n_CBF+n_controls] = joint_acc_lb[:7]
    ub[n_CBF:n_CBF+n_controls] = joint_acc_ub[:7]

    h_v_lb = v_EE[0:3] - v_EE_lb
    h_v_ub = v_EE_ub - v_EE[0:3]
    C[n_CBF+n_controls:n_CBF+n_controls+3, :] = J_EE[0:3,:]
    lb[n_CBF+n_controls:n_CBF+n_controls+3] = -20*h_v_lb - dJdq_EE[0:3]
    ub[n_CBF+n_controls:n_CBF+n_controls+3] = 20*h_v_ub - dJdq_EE[0:3]

    cbf_qp.update(g=g, C=C, l=lb, u=ub)
    cbf_qp.solve()
    ddq = cbf_qp.results.x

    # ddq = ddq_nominal
    tau = M @ ddq + Coriolis + static_friction * np.tanh(dq)

    tau = np.clip(tau, -20, 20)
    robot.setCommands(tau)

    ddq_list.append(ddq.copy())
    tau_list.append(tau.copy())
    all_h_list.append(all_h_np.copy())

robot.setCommands(np.zeros_like(tau))


### 4.3 Grasp the object

In [14]:
t_final = 2

# Get current pose
robot_info = robot.getJointStates()
q = robot_info['q'] # shape (7,)
dq = robot_info['dq'] # shape (7,)

q_pin = 0.30*np.ones(9)
dq_pin = np.zeros(9)
q_pin[0:n_joints] = q # shape (9,)
dq_pin[0:n_joints] = dq # shape (9,)

pin_info = pin_robot.getInfo(q_pin, dq_pin)
P_EE = pin_info["P_EE"]
R_EE = pin_info["R_EE"]

# Translational trajectory
P_EE_start = P_EE.copy()
P_EE_grasping = np.array([0.15, 0.62, 0.02])

via_points = np.array([P_EE_start, P_EE_grasping])
target_time = np.array([0, t_final])
Ts = 0.01
traj_position = PositionTrapezoidalTrajectory(via_points, target_time, T_antp=0.2, Ts=Ts)

# Rotational trajectory
R_EE_start  = R_EE.copy()

roll = np.pi
pitch = 0
yaw = np.pi/3
R_EE_pre_grasping = Rotation.from_euler('xyz', [roll, pitch, yaw]).as_matrix()

orientations = np.array([R_EE_start, R_EE_pre_grasping])
target_time = np.array([0, t_final])
traj_orientation = OrientationTrapezoidalTrajectory(orientations, target_time, Ts=Ts)

t_start = time.time()
while time.time() - t_start < t_final + 1:
    t = time.time() - t_start
    traj_pos, traj_pos_dt, traj_pos_dtdt = traj_position.get_traj_and_ders(t)
    traj_ori, traj_ori_dt, traj_ori_dtdt = traj_orientation.get_traj_and_ders(t)

    robot_info = robot.getJointStates()
    q = robot_info['q'] # shape (7,)
    dq = robot_info['dq'] # shape (7,)
    M = robot_info['M'] + 0.25*np.eye(n_joints) # shape (7,7)
    G = robot_info['G'] # shape (7,)
    Coriolis = robot_info['C'] # shape (7,)

    q_pin = 0.025*np.ones(9)
    dq_pin = np.zeros(9)
    q_pin[0:n_joints] = q # shape (9,)
    dq_pin[0:n_joints] = dq # shape (9,)

    pin_info = pin_robot.getInfo(q_pin, dq_pin)

    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)

    # Primary obejctive: tracking control
    e_pos = P_EE - traj_pos # shape (3,)
    e_pos_dt = v_EE[:3] - traj_pos_dt # shape (3,)
    v_dt = traj_pos_dtdt - K_p_pos @ e_pos - K_d_pos @ e_pos_dt

    e_rot = SO3(R_EE @ traj_ori.T).log() # shape (3,)
    e_rot_dt = v_EE[3:] - traj_ori_dt # shape (3,)
    omega_dt = traj_ori_dtdt -K_p_rot @ e_rot - K_d_rot @ e_rot_dt

    v_EE_dt_desired = np.concatenate([v_dt, omega_dt])
    S = J_EE
    S_pinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    S_null = (np.eye(len(q)) - S_pinv @ S)
    ddq_task = S_pinv @ (v_EE_dt_desired - dJdq_EE)

    # Secondary objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    q_bar = 1/2*(joint_ub+joint_lb)
    e_joint = W @ (q - q_bar)
    e_joint_dot = W @ dq
    ddq_nominal = ddq_task + S_null @ (- Kp_joint @ e_joint - Kd_joint @ e_joint_dot)

    # CBF
    C = np.zeros([n_in, n_controls])
    lb = np.zeros(n_in)
    ub = np.zeros(n_in)
    CBF_tmp = np.zeros(n_CBF)
    phi1_tmp = np.zeros(n_CBF)
    phi2_tmp = np.zeros(n_CBF)

    all_P_np = np.zeros([n_selected_BBs, 3])
    all_quat_np = np.zeros([n_selected_BBs, 4])
    all_J_np = np.zeros([n_selected_BBs, 6, 7])
    all_dJdq_np = np.zeros([n_selected_BBs, 6])

    for (ii, bb_key) in enumerate(selected_BBs):
        all_P_np[ii, :] = pin_info["P_"+bb_key]
        all_J_np[ii, :, :] = pin_info["J_"+bb_key][:,:7]
        all_quat_np[ii, :] = get_quat_from_rot_matrix(pin_info["R_"+bb_key])
        all_dJdq_np[ii, :] = pin_info["dJdq_"+bb_key][:7]

    all_h_np, _, _, all_phi1_np, all_actuation_np, all_lb_np, all_ub_np = \
        probs.getCBFConstraints(dq, all_P_np, all_quat_np, all_J_np, all_dJdq_np, alpha0, gamma1, gamma2, compensation)
    
    # CBF-QP constraints
    C[0:n_CBF,:] = all_actuation_np
    lb[0:n_CBF] = all_lb_np
    ub[0:n_CBF] = all_ub_np
    CBF_tmp = all_h_np
    phi1_tmp = all_phi1_np

    g = -ddq_nominal

    C[n_CBF:n_CBF+n_controls,:] = np.eye(n_controls)
    lb[n_CBF:n_CBF+n_controls] = joint_acc_lb[:7]
    ub[n_CBF:n_CBF+n_controls] = joint_acc_ub[:7]

    h_v_lb = v_EE[0:3] - v_EE_lb
    h_v_ub = v_EE_ub - v_EE[0:3]
    C[n_CBF+n_controls:n_CBF+n_controls+3, :] = J_EE[0:3,:]
    lb[n_CBF+n_controls:n_CBF+n_controls+3] = -20*h_v_lb - dJdq_EE[0:3]
    ub[n_CBF+n_controls:n_CBF+n_controls+3] = 20*h_v_ub - dJdq_EE[0:3]

    cbf_qp.update(g=g, C=C, l=lb, u=ub)
    cbf_qp.solve()
    ddq = cbf_qp.results.x

    # ddq = ddq_nominal
    tau = M @ ddq + Coriolis + static_friction * np.tanh(dq)

    tau = np.clip(tau, -20, 20)
    robot.setCommands(tau)

    ddq_list.append(ddq.copy())
    tau_list.append(tau.copy())
    all_h_list.append(all_h_np.copy())

robot.setCommands(np.zeros_like(tau))

# Grasp the object
gripper_width = 0.04  # m
speed = 0.05  # m / s
force = 10  # N
epsilon_inner = 0.01  # m
epsilon_outer = 0.01  # m
gripper.grasp(gripper_width, speed, force, epsilon_inner, epsilon_outer)


RuntimeError: libfranka gripper: Command failed!

### 4.4 Move out of the box

In [15]:
t_1 = 2
t_2 = 12
t_final = 22

# Get current pose
robot_info = robot.getJointStates()
q = robot_info['q'] # shape (7,)
dq = robot_info['dq'] # shape (7,)

q_pin = 0.30*np.ones(9)
dq_pin = np.zeros(9)
q_pin[0:n_joints] = q # shape (9,)
dq_pin[0:n_joints] = dq # shape (9,)

pin_info = pin_robot.getInfo(q_pin, dq_pin)
P_EE = pin_info["P_EE"]
R_EE = pin_info["R_EE"]

# Translational trajectory
P_EE_start = P_EE.copy()
P_EE_pre_grasping = np.array([0.15, 0.64, 0.10])
P_EE_end = np.array([0.50, 0.25, 0.08])

via_points = np.array([P_EE_start, P_EE_pre_grasping, P_EE_end, P_EE_end])
target_time = np.array([0, t_1, t_2, t_final])
Ts = 0.01
traj_position = PositionTrapezoidalTrajectory(via_points, target_time, T_antp=0.2, Ts=Ts)

# Rotational trajectory
R_EE_start  = R_EE.copy()

roll = np.pi
pitch = 0
yaw = np.pi/3
R_EE_pre_grasping = Rotation.from_euler('xyz', [roll, pitch, yaw]).as_matrix()

roll = np.pi
pitch = 0
yaw = np.pi/3
R_EE_end = Rotation.from_euler('xyz', [roll, pitch, yaw]).as_matrix()

orientations = np.array([R_EE_start, R_EE_pre_grasping, R_EE_end, R_EE_end])
target_time = np.array([0, t_1, t_2, t_final])
traj_orientation = OrientationTrapezoidalTrajectory(orientations, target_time, Ts=Ts)

t_start = time.time()
while time.time() - t_start < t_final + 1:
    t = time.time() - t_start
    traj_pos, traj_pos_dt, traj_pos_dtdt = traj_position.get_traj_and_ders(t)
    traj_ori, traj_ori_dt, traj_ori_dtdt = traj_orientation.get_traj_and_ders(t)

    robot_info = robot.getJointStates()
    q = robot_info['q'] # shape (7,)
    dq = robot_info['dq'] # shape (7,)
    M = robot_info['M'] + 0.25*np.eye(n_joints) # shape (7,7)
    G = robot_info['G'] # shape (7,)
    Coriolis = robot_info['C'] # shape (7,)

    q_pin = 0.025*np.ones(9)
    dq_pin = np.zeros(9)
    q_pin[0:n_joints] = q # shape (9,)
    dq_pin[0:n_joints] = dq # shape (9,)

    pin_info = pin_robot.getInfo(q_pin, dq_pin)

    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)

    # Primary obejctive: tracking control
    e_pos = P_EE - traj_pos # shape (3,)
    e_pos_dt = v_EE[:3] - traj_pos_dt # shape (3,)
    v_dt = traj_pos_dtdt - K_p_pos @ e_pos - K_d_pos @ e_pos_dt

    e_rot = SO3(R_EE @ traj_ori.T).log() # shape (3,)
    e_rot_dt = v_EE[3:] - traj_ori_dt # shape (3,)
    omega_dt = traj_ori_dtdt -K_p_rot @ e_rot - K_d_rot @ e_rot_dt

    v_EE_dt_desired = np.concatenate([v_dt, omega_dt])
    S = J_EE
    S_pinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    S_null = (np.eye(len(q)) - S_pinv @ S)
    ddq_task = S_pinv @ (v_EE_dt_desired - dJdq_EE)

    # Secondary objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    q_bar = 1/2*(joint_ub+joint_lb)
    e_joint = W @ (q - q_bar)
    e_joint_dot = W @ dq
    ddq_nominal = ddq_task + S_null @ (- Kp_joint @ e_joint - Kd_joint @ e_joint_dot)

    # CBF
    C = np.zeros([n_in, n_controls])
    lb = np.zeros(n_in)
    ub = np.zeros(n_in)
    CBF_tmp = np.zeros(n_CBF)
    phi1_tmp = np.zeros(n_CBF)
    phi2_tmp = np.zeros(n_CBF)

    all_P_np = np.zeros([n_selected_BBs, 3])
    all_quat_np = np.zeros([n_selected_BBs, 4])
    all_J_np = np.zeros([n_selected_BBs, 6, 7])
    all_dJdq_np = np.zeros([n_selected_BBs, 6])

    for (ii, bb_key) in enumerate(selected_BBs):
        all_P_np[ii, :] = pin_info["P_"+bb_key]
        all_J_np[ii, :, :] = pin_info["J_"+bb_key][:,:7]
        all_quat_np[ii, :] = get_quat_from_rot_matrix(pin_info["R_"+bb_key])
        all_dJdq_np[ii, :] = pin_info["dJdq_"+bb_key][:7]

    all_h_np, _, _, all_phi1_np, all_actuation_np, all_lb_np, all_ub_np = \
        probs.getCBFConstraints(dq, all_P_np, all_quat_np, all_J_np, all_dJdq_np, alpha0, gamma1, gamma2, compensation)
    
    # CBF-QP constraints
    C[0:n_CBF,:] = all_actuation_np
    lb[0:n_CBF] = all_lb_np
    ub[0:n_CBF] = all_ub_np
    CBF_tmp = all_h_np
    phi1_tmp = all_phi1_np

    g = -ddq_nominal

    C[n_CBF:n_CBF+n_controls,:] = np.eye(n_controls)
    lb[n_CBF:n_CBF+n_controls] = joint_acc_lb[:7]
    ub[n_CBF:n_CBF+n_controls] = joint_acc_ub[:7]

    h_v_lb = v_EE[0:3] - v_EE_lb
    h_v_ub = v_EE_ub - v_EE[0:3]
    C[n_CBF+n_controls:n_CBF+n_controls+3, :] = J_EE[0:3,:]
    lb[n_CBF+n_controls:n_CBF+n_controls+3] = -20*h_v_lb - dJdq_EE[0:3]
    ub[n_CBF+n_controls:n_CBF+n_controls+3] = 20*h_v_ub - dJdq_EE[0:3]

    cbf_qp.update(g=g, C=C, l=lb, u=ub)
    cbf_qp.solve()
    ddq = cbf_qp.results.x

    # ddq = ddq_nominal
    tau = M @ ddq + Coriolis + static_friction * np.tanh(dq)

    tau = np.clip(tau, -20, 20)
    robot.setCommands(tau)

    ddq_list.append(ddq.copy())
    tau_list.append(tau.copy())
    all_h_list.append(all_h_np.copy())


### 4.5 Drop the object

In [16]:
robot.setCommands(np.zeros_like(tau))
gripper_width = 0.08
speed = 0.05
gripper.move(gripper_width, speed)

True

## 5. Collision avoidance with circulation

### 5.1 Define the CBF-QP and various parameters

In [14]:
from cores.utils.trajectory_utils import PositionTrapezoidalTrajectory, OrientationTrapezoidalTrajectory

# Define proxuite problem
n_controls = 7
n_obstacle = n_polytope + 1
n_CBF = n_selected_BBs*n_obstacle
n_in = n_controls + 2 + 3
cbf_qp = init_proxsuite_qp(n_v=n_controls, n_eq=0, n_in=n_in)

# Define CBF parameters
f0 = 0.3
rho = 5
alpha0 = 1.03
gamma1 = 20
gamma2 = 20
compensation = 0

# Define tracking gains
K_p_pos = np.diag([100,100,100])*0.9
K_d_pos = np.diag([50,50,50])*0.5

K_p_rot = np.diag([200,200,200])*1.8
K_d_rot = np.diag([100,100,100])*0.4

Kp_joint = 20*np.diag([1, 1, 1, 1, 1, 1, 1])
Kd_joint = 10*np.diag([1, 1, 1, 1, 1, 1, 1])

# Define the record
q_list = []
dq_list = []
ddq_list = []
tau_list = []
all_h_list = []
loop_time_list = []

### 5.2 Move from the starting point to pre-grasping pose 

In [15]:
t_final = 10

# Get current pose
robot_info = robot.getJointStates()
q = robot_info['q'] # shape (7,)
dq = robot_info['dq'] # shape (7,)

q_pin = 0.30*np.ones(9)
dq_pin = np.zeros(9)
q_pin[0:n_joints] = q # shape (9,)
dq_pin[0:n_joints] = dq # shape (9,)

pin_info = pin_robot.getInfo(q_pin, dq_pin)
P_EE = pin_info["P_EE"]
R_EE = pin_info["R_EE"]

# Translational trajectory
P_EE_start = P_EE.copy()
P_EE_pre_grasping = np.array([0.15, 0.62, 0.10])

via_points = np.array([P_EE_start, P_EE_pre_grasping])
target_time = np.array([0, t_final])
Ts = 0.01
traj_position = PositionTrapezoidalTrajectory(via_points, target_time, T_antp=0.2, Ts=Ts)

# Rotational trajectory
R_EE_start  = R_EE.copy()

roll = np.pi
pitch = 0
yaw = np.pi/3
R_EE_pre_grasping = Rotation.from_euler('xyz', [roll, pitch, yaw]).as_matrix()

orientations = np.array([R_EE_start, R_EE_pre_grasping])
target_time = np.array([0, t_final])
traj_orientation = OrientationTrapezoidalTrajectory(orientations, target_time, Ts=Ts)

t_start = time.time()
while time.time() - t_start < t_final + 1:
    t_loop_start = time.time()
    t = time.time() - t_start
    traj_pos, traj_pos_dt, traj_pos_dtdt = traj_position.get_traj_and_ders(t)
    traj_ori, traj_ori_dt, traj_ori_dtdt = traj_orientation.get_traj_and_ders(t)

    robot_info = robot.getJointStates()
    q = robot_info['q'] # shape (7,)
    dq = robot_info['dq'] # shape (7,)
    M = robot_info['M'] + np.diag([0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.2]) # shape (7,7)
    G = robot_info['G'] # shape (7,)
    Coriolis = robot_info['C'] # shape (7,)

    q_pin = 0.025*np.ones(9)
    dq_pin = np.zeros(9)
    q_pin[0:n_joints] = q # shape (9,)
    dq_pin[0:n_joints] = dq # shape (9,)

    pin_info = pin_robot.getInfo(q_pin, dq_pin)

    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)2

    # Primary obejctive: tracking control
    e_pos = P_EE - traj_pos # shape (3,)
    e_pos_dt = v_EE[:3] - traj_pos_dt # shape (3,)
    v_dt = traj_pos_dtdt - K_p_pos @ e_pos - K_d_pos @ e_pos_dt

    e_rot = SO3(R_EE @ traj_ori.T).log() # shape (3,)
    e_rot_dt = v_EE[3:] - traj_ori_dt # shape (3,)
    omega_dt = traj_ori_dtdt -K_p_rot @ e_rot - K_d_rot @ e_rot_dt

    v_EE_dt_desired = np.concatenate([v_dt, omega_dt])
    S = J_EE
    S_pinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    S_null = (np.eye(len(q)) - S_pinv @ S)
    ddq_task = S_pinv @ (v_EE_dt_desired - dJdq_EE)

    # Secondary objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    q_bar = 1/2*(joint_ub+joint_lb)
    e_joint = W @ (q - q_bar)
    e_joint_dot = W @ dq
    ddq_nominal = ddq_task + S_null @ (- Kp_joint @ e_joint - Kd_joint @ e_joint_dot)

    C = np.zeros([n_in, n_controls])
    lb = np.zeros(n_in)
    ub = np.zeros(n_in)
    CBF_tmp = np.zeros(n_CBF)
    smooth_min_tmp = 0
    phi1_tmp = 0
    phi2_tmp = 0

    all_P_np = np.zeros([n_selected_BBs, 3])
    all_quat_np = np.zeros([n_selected_BBs, 4])
    all_J_np = np.zeros([n_selected_BBs, 6, 7])
    all_dJdq_np = np.zeros([n_selected_BBs, 6])

    for (ii, bb_key) in enumerate(selected_BBs):
        all_P_np[ii, :] = pin_info["P_"+bb_key]
        all_J_np[ii, :, :] = pin_info["J_"+bb_key][:,:7]
        all_quat_np[ii, :] = get_quat_from_rot_matrix(pin_info["R_"+bb_key])
        all_dJdq_np[ii, :] = pin_info["dJdq_"+bb_key][:7]

    all_h_np, _, _, first_order_all_average_scalar_np, second_order_all_average_scalar_np, second_order_all_average_vector_np = \
            probs.getSmoothMinCBFConstraints(dq, all_P_np, all_quat_np, all_J_np, all_dJdq_np, alpha0)
    
    min_h = np.min(all_h_np)
    indices = np.where(rho*(all_h_np - min_h) < 708)[0]
    h_selected = all_h_np[indices]
    first_order_all_average_scalar_selected = first_order_all_average_scalar_np[indices]
    second_order_all_average_scalar_selected = second_order_all_average_scalar_np[indices]
    second_order_all_average_vector_selected = second_order_all_average_vector_np[indices,:]

    f, f_dh, f_dhdh = sfh.getSmoothMinimumAndLocalGradientAndHessian(rho, h_selected)

    # CBF-QP constraints
    CBF = f - f0
    # print(CBF)
    dCBF =  f_dh @ first_order_all_average_scalar_selected # scalar
    phi1 = dCBF + gamma1 * CBF

    C[0,:] = f_dh @ second_order_all_average_vector_selected
    lb[0] = - gamma2*phi1 - gamma1*dCBF - first_order_all_average_scalar_selected.T @ f_dhdh @ first_order_all_average_scalar_selected \
            - f_dh @ second_order_all_average_scalar_selected + compensation
    ub[0] = np.inf

    # Circulation constraints
    tmp = np.array([0, -C[0,2], C[0,1], -C[0,4], C[0,3], -C[0,6], C[0,5]])
    C[1,:] = tmp
    ueq = np.zeros_like(ddq_nominal)
    threshold = 2e-2
    lb[1] = C[1,:] @ ueq + 590*(1-np.exp(2*(CBF-threshold)))
    ub[1] = np.inf

    h_v_lb = v_EE[0:3] - v_EE_lb
    h_v_ub = v_EE_ub - v_EE[0:3]
    C[2+n_controls:5+n_controls,:] = J_EE[0:3,:]
    lb[2+n_controls:5+n_controls] = -20*h_v_lb - dJdq_EE[0:3]
    ub[2+n_controls:5+n_controls] = 20*h_v_ub - dJdq_EE[0:3]


    # CBF-QP constraints
    # print(np.min(CBF_tmp))
    g = -ddq_nominal

    C[2:2+n_controls,:] = np.eye(n_controls)
    lb[2:2+n_controls] = joint_acc_lb[:7]
    ub[2:2+n_controls] = joint_acc_ub[:7]


    cbf_qp.update(g=g, C=C, l=lb, u=ub)
    cbf_qp.solve()
    ddq = cbf_qp.results.x
    # print(cbf_qp.results.info.status)

    # tau = M_pin @ ddq + nle_pin - G_pin + static_friction * np.tanh(dq_moving_avg)
    tau = M @ ddq + Coriolis + static_friction * np.tanh(dq)

    tau = np.clip(tau, -30, 30)
    robot.setCommands(tau)
    t_loop_end = time.time()

    q_list.append(q.copy())
    dq_list.append(dq.copy())
    ddq_list.append(ddq.copy())
    tau_list.append(tau.copy())
    all_h_list.append(all_h_np.copy())
    loop_time_list.append(t_loop_end - t_loop_start)

robot.setCommands(np.zeros_like(tau))

### 5.3 Grasp the object

In [16]:
t_final = 2

# Get current pose
robot_info = robot.getJointStates()
q = robot_info['q'] # shape (7,)
dq = robot_info['dq'] # shape (7,)

q_pin = 0.30*np.ones(9)
dq_pin = np.zeros(9)
q_pin[0:n_joints] = q # shape (9,)
dq_pin[0:n_joints] = dq # shape (9,)

pin_info = pin_robot.getInfo(q_pin, dq_pin)
P_EE = pin_info["P_EE"]
R_EE = pin_info["R_EE"]

# Translational trajectory
P_EE_start = P_EE.copy()
P_EE_grasping = np.array([0.15, 0.62, 0.02])

via_points = np.array([P_EE_start, P_EE_grasping])
target_time = np.array([0, t_final])
Ts = 0.01
traj_position = PositionTrapezoidalTrajectory(via_points, target_time, T_antp=0.2, Ts=Ts)

# Rotational trajectory
R_EE_start  = R_EE.copy()

roll = np.pi
pitch = 0
yaw = np.pi/3
R_EE_pre_grasping = Rotation.from_euler('xyz', [roll, pitch, yaw]).as_matrix()

orientations = np.array([R_EE_start, R_EE_pre_grasping])
target_time = np.array([0, t_final])
traj_orientation = OrientationTrapezoidalTrajectory(orientations, target_time, Ts=Ts)

t_start = time.time()
while time.time() - t_start < t_final + 1:
    t_loop_start = time.time()
    t = time.time() - t_start
    traj_pos, traj_pos_dt, traj_pos_dtdt = traj_position.get_traj_and_ders(t)
    traj_ori, traj_ori_dt, traj_ori_dtdt = traj_orientation.get_traj_and_ders(t)

    robot_info = robot.getJointStates()
    q = robot_info['q'] # shape (7,)
    dq = robot_info['dq'] # shape (7,)
    M = robot_info['M'] + np.diag([0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.2]) # shape (7,7)
    G = robot_info['G'] # shape (7,)
    Coriolis = robot_info['C'] # shape (7,)

    q_pin = 0.025*np.ones(9)
    dq_pin = np.zeros(9)
    q_pin[0:n_joints] = q # shape (9,)
    dq_pin[0:n_joints] = dq # shape (9,)

    pin_info = pin_robot.getInfo(q_pin, dq_pin)

    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)2

    # Primary obejctive: tracking control
    e_pos = P_EE - traj_pos # shape (3,)
    e_pos_dt = v_EE[:3] - traj_pos_dt # shape (3,)
    v_dt = traj_pos_dtdt - K_p_pos @ e_pos - K_d_pos @ e_pos_dt

    e_rot = SO3(R_EE @ traj_ori.T).log() # shape (3,)
    e_rot_dt = v_EE[3:] - traj_ori_dt # shape (3,)
    omega_dt = traj_ori_dtdt -K_p_rot @ e_rot - K_d_rot @ e_rot_dt

    v_EE_dt_desired = np.concatenate([v_dt, omega_dt])
    S = J_EE
    S_pinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    S_null = (np.eye(len(q)) - S_pinv @ S)
    ddq_task = S_pinv @ (v_EE_dt_desired - dJdq_EE)

    # Secondary objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    q_bar = 1/2*(joint_ub+joint_lb)
    e_joint = W @ (q - q_bar)
    e_joint_dot = W @ dq
    ddq_nominal = ddq_task + S_null @ (- Kp_joint @ e_joint - Kd_joint @ e_joint_dot)

    C = np.zeros([n_in, n_controls])
    lb = np.zeros(n_in)
    ub = np.zeros(n_in)
    CBF_tmp = np.zeros(n_CBF)
    smooth_min_tmp = 0
    phi1_tmp = 0
    phi2_tmp = 0

    all_P_np = np.zeros([n_selected_BBs, 3])
    all_quat_np = np.zeros([n_selected_BBs, 4])
    all_J_np = np.zeros([n_selected_BBs, 6, 7])
    all_dJdq_np = np.zeros([n_selected_BBs, 6])

    for (ii, bb_key) in enumerate(selected_BBs):
        all_P_np[ii, :] = pin_info["P_"+bb_key]
        all_J_np[ii, :, :] = pin_info["J_"+bb_key][:,:7]
        all_quat_np[ii, :] = get_quat_from_rot_matrix(pin_info["R_"+bb_key])
        all_dJdq_np[ii, :] = pin_info["dJdq_"+bb_key][:7]

    all_h_np, _, _, first_order_all_average_scalar_np, second_order_all_average_scalar_np, second_order_all_average_vector_np = \
            probs.getSmoothMinCBFConstraints(dq, all_P_np, all_quat_np, all_J_np, all_dJdq_np, alpha0)
    
    min_h = np.min(all_h_np)
    indices = np.where(rho*(all_h_np - min_h) < 708)[0]
    h_selected = all_h_np[indices]
    first_order_all_average_scalar_selected = first_order_all_average_scalar_np[indices]
    second_order_all_average_scalar_selected = second_order_all_average_scalar_np[indices]
    second_order_all_average_vector_selected = second_order_all_average_vector_np[indices,:]

    f, f_dh, f_dhdh = sfh.getSmoothMinimumAndLocalGradientAndHessian(rho, h_selected)

    # CBF-QP constraints
    CBF = f - f0
    # print(CBF)
    dCBF =  f_dh @ first_order_all_average_scalar_selected # scalar
    phi1 = dCBF + gamma1 * CBF

    C[0,:] = f_dh @ second_order_all_average_vector_selected
    lb[0] = - gamma2*phi1 - gamma1*dCBF - first_order_all_average_scalar_selected.T @ f_dhdh @ first_order_all_average_scalar_selected \
            - f_dh @ second_order_all_average_scalar_selected + compensation
    ub[0] = np.inf

    # Circulation constraints
    # tmp = np.array([0, -C[0,2], C[0,1], -C[0,4], C[0,3], -C[0,6], C[0,5]])
    # C[1,:] = tmp
    # ueq = np.zeros_like(ddq_nominal)
    # threshold = 2e-2
    # lb[1] = C[1,:] @ ueq + 590*(1-np.exp(2*(CBF-threshold)))
    # ub[1] = np.inf

    h_v_lb = v_EE[0:3] - v_EE_lb
    h_v_ub = v_EE_ub - v_EE[0:3]
    C[2+n_controls:5+n_controls,:] = J_EE[0:3,:]
    lb[2+n_controls:5+n_controls] = -20*h_v_lb - dJdq_EE[0:3]
    ub[2+n_controls:5+n_controls] = 20*h_v_ub - dJdq_EE[0:3]


    # CBF-QP constraints
    # print(np.min(CBF_tmp))
    g = -ddq_nominal

    C[2:2+n_controls,:] = np.eye(n_controls)
    lb[2:2+n_controls] = joint_acc_lb[:7]
    ub[2:2+n_controls] = joint_acc_ub[:7]

    cbf_qp.update(g=g, C=C, l=lb, u=ub)
    cbf_qp.solve()
    ddq = cbf_qp.results.x
    # print(cbf_qp.results.info.status)

    # tau = M_pin @ ddq + nle_pin - G_pin + static_friction * np.tanh(dq_moving_avg)
    tau = M @ ddq + Coriolis + static_friction * np.tanh(dq)

    tau = np.clip(tau, -30, 30)
    robot.setCommands(tau)
    t_loop_end = time.time()

    q_list.append(q.copy())
    dq_list.append(dq.copy())
    ddq_list.append(ddq.copy())
    tau_list.append(tau.copy())
    all_h_list.append(all_h_np.copy())
    loop_time_list.append(t_loop_end - t_loop_start)

robot.setCommands(np.zeros_like(tau))

# Grasp the object
gripper_width = 0.04  # m
speed = 0.05  # m / s
force = 10  # N
epsilon_inner = 0.01  # m
epsilon_outer = 0.01  # m
gripper.grasp(gripper_width, speed, force, epsilon_inner, epsilon_outer)

True

### 5.4 Move out of the box

In [19]:
t_1 = 2
t_2 = 12
t_final = 22

# Get current pose
robot_info = robot.getJointStates()
q = robot_info['q'] # shape (7,)
dq = robot_info['dq'] # shape (7,)

q_pin = 0.30*np.ones(9)
dq_pin = np.zeros(9)
q_pin[0:n_joints] = q # shape (9,)
dq_pin[0:n_joints] = dq # shape (9,)

pin_info = pin_robot.getInfo(q_pin, dq_pin)
P_EE = pin_info["P_EE"]
R_EE = pin_info["R_EE"]

# Translational trajectory
P_EE_start = P_EE.copy()
P_EE_pre_grasping = np.array([0.15, 0.62, 0.10])
P_EE_end = np.array([0.50, 0.25, 0.08])

via_points = np.array([P_EE_start, P_EE_pre_grasping, P_EE_end, P_EE_end])
target_time = np.array([0, t_1, t_2, t_final])
Ts = 0.01
traj_position = PositionTrapezoidalTrajectory(via_points, target_time, T_antp=0.2, Ts=Ts)

# Rotational trajectory
R_EE_start  = R_EE.copy()

roll = np.pi
pitch = 0
yaw = np.pi/3
R_EE_pre_grasping = Rotation.from_euler('xyz', [roll, pitch, yaw]).as_matrix()

roll = np.pi
pitch = 0
yaw = np.pi/3
R_EE_end = Rotation.from_euler('xyz', [roll, pitch, yaw]).as_matrix()

orientations = np.array([R_EE_start, R_EE_pre_grasping, R_EE_end, R_EE_end])
target_time = np.array([0, t_1, t_2, t_final])
traj_orientation = OrientationTrapezoidalTrajectory(orientations, target_time, Ts=Ts)

t_start = time.time()
while time.time() - t_start < t_final + 1:
    t_loop_start = time.time()
    t = time.time() - t_start
    traj_pos, traj_pos_dt, traj_pos_dtdt = traj_position.get_traj_and_ders(t)
    traj_ori, traj_ori_dt, traj_ori_dtdt = traj_orientation.get_traj_and_ders(t)

    robot_info = robot.getJointStates()
    q = robot_info['q'] # shape (7,)
    dq = robot_info['dq'] # shape (7,)
    M = robot_info['M'] + np.diag([0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.20]) # shape (7,7)
    G = robot_info['G'] # shape (7,)
    Coriolis = robot_info['C'] # shape (7,)

    q_pin = 0.025*np.ones(9)
    dq_pin = np.zeros(9)
    q_pin[0:n_joints] = q # shape (9,)
    dq_pin[0:n_joints] = dq # shape (9,)

    pin_info = pin_robot.getInfo(q_pin, dq_pin)

    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)2

    # Primary obejctive: tracking control
    e_pos = P_EE - traj_pos # shape (3,)
    e_pos_dt = v_EE[:3] - traj_pos_dt # shape (3,)
    v_dt = traj_pos_dtdt - K_p_pos @ e_pos - K_d_pos @ e_pos_dt

    e_rot = SO3(R_EE @ traj_ori.T).log() # shape (3,)
    e_rot_dt = v_EE[3:] - traj_ori_dt # shape (3,)
    omega_dt = traj_ori_dtdt -K_p_rot @ e_rot - K_d_rot @ e_rot_dt

    v_EE_dt_desired = np.concatenate([v_dt, omega_dt])
    S = J_EE
    S_pinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    S_null = (np.eye(len(q)) - S_pinv @ S)
    ddq_task = S_pinv @ (v_EE_dt_desired - dJdq_EE)

    # Secondary objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    q_bar = 1/2*(joint_ub+joint_lb)
    e_joint = W @ (q - q_bar)
    e_joint_dot = W @ dq
    ddq_nominal = ddq_task + S_null @ (- Kp_joint @ e_joint - Kd_joint @ e_joint_dot)

    C = np.zeros([n_in, n_controls])
    lb = np.zeros(n_in)
    ub = np.zeros(n_in)
    CBF_tmp = np.zeros(n_CBF)
    smooth_min_tmp = 0
    phi1_tmp = 0
    phi2_tmp = 0

    all_P_np = np.zeros([n_selected_BBs, 3])
    all_quat_np = np.zeros([n_selected_BBs, 4])
    all_J_np = np.zeros([n_selected_BBs, 6, 7])
    all_dJdq_np = np.zeros([n_selected_BBs, 6])

    for (ii, bb_key) in enumerate(selected_BBs):
        all_P_np[ii, :] = pin_info["P_"+bb_key]
        all_J_np[ii, :, :] = pin_info["J_"+bb_key][:,:7]
        all_quat_np[ii, :] = get_quat_from_rot_matrix(pin_info["R_"+bb_key])
        all_dJdq_np[ii, :] = pin_info["dJdq_"+bb_key][:7]

    all_h_np, _, _, first_order_all_average_scalar_np, second_order_all_average_scalar_np, second_order_all_average_vector_np = \
            probs.getSmoothMinCBFConstraints(dq, all_P_np, all_quat_np, all_J_np, all_dJdq_np, alpha0)
    
    min_h = np.min(all_h_np)
    indices = np.where(rho*(all_h_np - min_h) < 708)[0]
    h_selected = all_h_np[indices]
    first_order_all_average_scalar_selected = first_order_all_average_scalar_np[indices]
    second_order_all_average_scalar_selected = second_order_all_average_scalar_np[indices]
    second_order_all_average_vector_selected = second_order_all_average_vector_np[indices,:]

    f, f_dh, f_dhdh = sfh.getSmoothMinimumAndLocalGradientAndHessian(rho, h_selected)

    # CBF-QP constraints
    CBF = f - f0
    # print(CBF)
    dCBF =  f_dh @ first_order_all_average_scalar_selected # scalar
    phi1 = dCBF + gamma1 * CBF

    C[0,:] = f_dh @ second_order_all_average_vector_selected
    lb[0] = - gamma2*phi1 - gamma1*dCBF - first_order_all_average_scalar_selected.T @ f_dhdh @ first_order_all_average_scalar_selected \
            - f_dh @ second_order_all_average_scalar_selected + compensation
    ub[0] = np.inf

    # Circulation constraints
    if t > t_1:
        tmp = np.array([0, -C[0,2], C[0,1], -C[0,4], C[0,3], -C[0,6], C[0,5]])
        C[1,:] = tmp
        ueq = np.zeros_like(ddq_nominal)
        threshold = 2e-2
        lb[1] = C[1,:] @ ueq + 615*(1-np.exp(2*(CBF-threshold)))
        ub[1] = np.inf

    h_v_lb = v_EE[0:3] - v_EE_lb
    h_v_ub = v_EE_ub - v_EE[0:3]
    C[2+n_controls:5+n_controls,:] = J_EE[0:3,:]
    lb[2+n_controls:5+n_controls] = -20*h_v_lb - dJdq_EE[0:3]
    ub[2+n_controls:5+n_controls] = 20*h_v_ub - dJdq_EE[0:3]


    # CBF-QP constraints
    # print(np.min(CBF_tmp))
    g = -ddq_nominal

    C[2:2+n_controls,:] = np.eye(n_controls)
    lb[2:2+n_controls] = joint_acc_lb[:7]
    ub[2:2+n_controls] = joint_acc_ub[:7]

    cbf_qp.update(g=g, C=C, l=lb, u=ub)
    cbf_qp.solve()
    ddq = cbf_qp.results.x
    # print(cbf_qp.results.info.status)

    # tau = M_pin @ ddq + nle_pin - G_pin + static_friction * np.tanh(dq_moving_avg)
    tau = M @ ddq + Coriolis + static_friction * np.tanh(dq)

    tau = np.clip(tau, -30, 30)
    robot.setCommands(tau)
    t_loop_end = time.time()

    q_list.append(q.copy())
    dq_list.append(dq.copy())
    ddq_list.append(ddq.copy())
    tau_list.append(tau.copy())
    all_h_list.append(all_h_np.copy())
    loop_time_list.append(t_loop_end - t_loop_start)

robot.setCommands(np.zeros_like(tau))


### 5.5 Drop the ball

In [20]:
robot.setCommands(np.zeros_like(tau))
gripper_width = 0.08
speed = 0.05
gripper.move(gripper_width, speed)

True